In [1]:
#%pip install einops transformers sae_lens sae_vis
import einops
import os
from utils_folder import dashboards
from transformers import AutoModel, AutoTokenizer
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes, convert_connor_rob_sae_to_our_saelens_format, download_sae_from_hf
import numpy as np
from matplotlib.colors import LogNorm, Normalize
import pandas as pd
import seaborn as sns
import torch as t
import plotly_express as px
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
from scipy import stats
from torch.utils.data import Dataset
from sae_vis.model_fns import AutoEncoderConfig, AutoEncoder
from transformer_lens import utils as tl_utils
from transformer_lens.hook_points import (
    HookPoint,
)

from transformer_lens import HookedTransformer, HookedTransformerConfig
from transformer_lens.utils import (
    load_dataset,
    tokenize_and_concatenate,
    download_file_from_hf,
)

from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
device = t.device("cuda" if t.cuda.is_available() else "cpu")
from transformer_lens import HookedSAE, HookedSAEConfig
from transformer_lens.utils import download_file_from_hf


from importlib import reload
from utils_folder.autoencoder import AutoEncoder
import matplotlib as mpl
cmap = mpl.colormaps["Spectral"]

from utils_folder import SAE_metrics
from utils_folder import vector_metrics
from utils_folder import misc_utils

import torch 


/home/ubuntu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-05-24 21:07:44.549956: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-24 21:07:44.592156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
gelu2 = HookedTransformer.from_pretrained("gelu-2l").to(device)
gpt_small = HookedTransformer.from_pretrained("gpt2-small").to(device)

Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cuda


/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


In [7]:
logits, cache = gpt_small.run_with_cache('today I went to the beach and I saw')

In [31]:
attn_patterns = [cache[k] for k in cache if 'attn.hook_pattern' in k] # layer head query key
attn[0].shape

torch.Size([1, 12, 10, 10])

In [57]:
contribution1

tensor([[1.7586e-01, 6.1720e-02, 9.7753e-02, 1.0556e-01, 1.2122e-01, 1.0165e-01,
         4.5974e-02, 1.4088e-01, 1.4243e-01, 6.9495e-03],
        [4.6450e-01, 5.6798e-02, 6.1003e-02, 4.7216e-02, 7.8358e-02, 8.2871e-02,
         4.2495e-02, 1.2238e-01, 4.1550e-02, 2.8317e-03],
        [4.5900e-01, 2.0960e-02, 9.2207e-02, 7.3440e-02, 1.1960e-01, 7.3603e-02,
         3.3193e-02, 8.0797e-02, 3.0337e-02, 1.6855e-02],
        [7.3982e-01, 5.6579e-02, 4.7204e-02, 4.3899e-02, 3.1680e-02, 2.4851e-02,
         2.1732e-02, 1.7152e-02, 1.0241e-02, 6.8400e-03],
        [7.2622e-01, 5.5624e-02, 3.4963e-02, 4.7602e-02, 3.1507e-02, 2.7072e-02,
         3.3840e-02, 1.7616e-02, 1.1864e-02, 1.3690e-02],
        [7.7203e-01, 3.4316e-02, 5.9629e-02, 1.9105e-02, 4.4625e-02, 2.0528e-02,
         4.0044e-03, 3.0037e-02, 9.6162e-03, 6.1102e-03],
        [9.0993e-01, 2.3547e-02, 2.1805e-02, 1.2051e-02, 7.0969e-03, 8.8852e-03,
         7.3795e-03, 2.9199e-03, 2.2177e-03, 4.1655e-03],
        [8.2964e-01, 4.3540

In [73]:
for layer in range(len(attn_patterns)):
    attn_pattern = attn_patterns[0][0,:,:,:]
    contribution = attn_pattern[:,-1,:]
    contribution = contribution.mean(dim=0)

    cumm_attn = 
    for prev_layer in range(layer-1):
        cumm_attn = 


    contribution_so_far = contribution 
    

attn_pattern1 = attn_patterns[1][0,:,:,:]
contribution1 = attn_pattern1[:,-1,:]
contribution1 = einops.einsum(contribution1, attn_pattern1, 'n_heads new_token, n_heads new_token old_token -> n_heads old_token')
contribution1 = contribution1.mean(dim=0)

contribution_so_far = contribution_so_far + contribution1


attn_pattern2 = attn_patterns[2][0,:,:,:]
contribution2 = attn_pattern2[:,-1,:]
contribution2 = einops.einsum(contribution2, attn_pattern2, 'n_heads new_token, n_heads new_token old_token -> n_heads old_token')
contribution2 = einops.einsum(contribution2, attn_pattern1, 'n_heads new_token, n_heads new_token old_token -> n_heads old_token')
contribution2 = contribution2.mean(dim=0)


contribution_so_far = contribution_so_far + contribution2



In [75]:
total_cont = contribution_so_far/contribution_so_far.sum()

In [71]:
total_cont = contribution_so_far/contribution_so_far.sum()

In [72]:
total_cont

tensor([0.3822, 0.0595, 0.0514, 0.0526, 0.0526, 0.0444, 0.0344, 0.0563, 0.0469,
        0.2197], device='cuda:0')